In [ ]:
import math
import functools

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('expression_counts.txt', sep=' ')
df.info()
df.head()

In [ ]:
df_filter = df[df > 10].dropna(axis=1)
df_filter.info()
df_filter

In [ ]:
metadata = pd.DataFrame({
    'condition': ['X'] * 6,
    'group': ['disease'] * 3 + ['control'] * 3
}, index=df.index)
metadata.info()
metadata

In [ ]:
dds = DeseqDataSet(
    counts=df_filter,
    metadata=metadata,
    design_factors="group",
    refit_cooks=True,
    n_cpus=8,
)
dds

In [ ]:
dds.deseq2()

In [ ]:
print(dds)

In [ ]:
stat_res = DeseqStats(dds, n_cpus=8, contrast=('group', 'control', 'disease'))
stat_res.summary()
res_df = stat_res.results_df.copy()
res_df['-log10(padj)'] = res_df['padj'].apply(lambda v: -math.log10(v))

In [ ]:
pv_lim = 0.05
lfc_lim = 0.6

fres = res_df[res_df['padj'] < pv_lim]

upr = fres[fres['log2FoldChange'] >= lfc_lim]
downr = fres[fres['log2FoldChange'] <= -lfc_lim]
no = fres[(fres['log2FoldChange'] < lfc_lim) & (fres['log2FoldChange'] > -lfc_lim)]

fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=upr['log2FoldChange'], y=upr['-log10(padj)'], name='UP'))
fig.add_trace(go.Scatter(mode='markers', x=downr['log2FoldChange'], y=downr['-log10(padj)'], name='DOWN'))
fig.add_trace(go.Scatter(mode='markers', x=no['log2FoldChange'], y=no['-log10(padj)'], name='UP'))

fig.add_vline(x=-lfc_lim, line_color='red')
fig.add_vline(x=lfc_lim, line_color='red')
fig.add_hline(y=-math.log10(pv_lim), line_color='red')

fig.update_layout(height=500)

fig.show()

In [ ]:
big = fres[fres['log2FoldChange'].abs() > lfc_lim]
big.info()

In [ ]:
extract = np.log2(df_filter[big.index.tolist()])
fig = px.imshow(extract)
fig.update_layout(height=200)
fig.show()

In [ ]:
df_t = df.T
df_t.info()
df_t.head()

In [ ]:
df_g = pd.DataFrame({
    'gene': df_t.index,
    'disease_mean': df_t[['disease_1', 'disease_2', 'disease_3']].mean(axis=1).reset_index(drop=True),
    'control_mean': df_t[['control_1', 'control_2', 'control_3']].mean(axis=1).reset_index(drop=True),
})
df_g.info()
df_g.head()

In [ ]:
df_g['fold_change'] = df_g['disease_mean'] / df_g['control_mean']
df_g['log2_fold_change'] = df_g['fold_change'].apply(lambda v: math.log(v, 2))
df_g.info()
df_g.head()

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=['Fold Change Distribution', 'Log2FoldChange Distribution'])
fig.add_trace(go.Histogram(x=df_g['fold_change']), row=1, col=1)
fig.add_trace(go.Histogram(x=df_g['log2_fold_change']), row=2, col=1)
fig.update_layout(height=500)
fig.show()